<a href="https://colab.research.google.com/github/Esantomi/digital-humanities/blob/main/lectures/%EC%9D%B8%EB%AC%B8%EC%A7%80%EC%8B%9D%EC%B2%98%EB%A6%AC%EC%99%80%ED%86%B5%EA%B3%84/03_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EA%B8%B0%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 환경 준비

!pip install konlpy
import pandas as pd

from IPython.display import clear_output
clear_output()

print("###준비가 완료되었습니다.###")

###준비가 완료되었습니다.###


In [ ]:
#@title 데이터 불러오기(탭 \t 으로 분리)
구글드라이브ID = '1VEdcJErjAjj2ul5gnS0-IgKrpGy0NEAK'  #@param {type: "string"}
파일저장이름 = 'rawdata.txt'  #@param {type: "string"}

import time
start = time.time() # 시작시간

# 구글 드라이브 연결을 위한 기본 세팅
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 문서 ID로 실제 파일 불러오는 법
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
rawdata_downloaded = drive.CreateFile({'id': 구글드라이브ID})
rawdata_downloaded.GetContentFile(파일저장이름)

print(파일저장이름+"을 정상적으로 가져왔습니다.")

import pandas as pd
# 파일을 DataFrame으로 전환
rawdata = pd.read_csv(파일저장이름, sep="\t", header=0, encoding='utf-8', engine='python')
rawdata.head()

rawdata.txt을 정상적으로 가져왔습니다.


,id,type,text
0,1,노무현,존경하는 국민 여러분. 오늘 저는 대한민국의 제16대 대통령에 취임하기 위해 이...
1,2,이명박,"존경하는 국민 여러분! 700만 해외동포 여러분, 이 자리에 참석하신 노무현, 김..."
2,3,박근혜,희망의 새 시대를 열겠습니다. 존경하는 국민 여러분! 700만 해외동포 여러분! ...
3,4,문재인,"존경하고 사랑하는 국민 여러분, 감사합니다. 국민 여러분의 위대한 선택에 머리 숙여..."


In [ ]:
#@title 형태소 분석 및 저장(파이썬은 0부터 시작!!!)
텍스트데이터위치 = 2  #@param {type: "number"}
타입데이터위치 = 1  #@param {type: "number"}
파일저장이름 = 'fin_wordcount.csv'  #@param {type: "string"}

# 분석하기
from itertools import product
import numpy as np
from konlpy.tag import Okt
okt = Okt()

segments  = []
for index, row in rawdata.iterrows():
    textdata = row[텍스트데이터위치]
    type = row[타입데이터위치]
    words = list(okt.pos(textdata))
    words = pd.DataFrame.from_records(words)
    words['type'] = type
    words['count'] = 1      
    segments.append(words)

# 단일 데이터프레임 구조로 변환
framemerge = pd.concat(segments)

# 데이터프레임의 컬럼값 조정
framemerge.columns = ["terms", "pos", "type", "count"]

print("##################################################")
print("타입별 형태소 추출 결과")
print(framemerge)
print("##################################################")

# 데이터 정리하기
dfwordflag = framemerge.groupby(['terms', 'pos', "type"])['count'].sum()

# 단일 데이터프레임 구조로 변환
dfwordflag = pd.DataFrame(dfwordflag)

print("##################################################")
print("형태소+품사+타입을 기준으로 묶어주기 결과")
print(dfwordflag)
print("##################################################")


#csv로 출력
dfwordflag.to_csv(파일저장이름,header='true', quotechar='"', encoding='utf-8')

print("##################################################")
print("데이터를 " + 파일저장이름 + "로 저장 완료")
print("##################################################")


##################################################
타입별 형태소 추출 결과
      terms          pos type  count
0        존경         Noun  노무현      1
1        하는         Verb  노무현      1
2        국민         Noun  노무현      1
3       여러분         Noun  노무현      1
4         .  Punctuation  노무현      1
...     ...          ...  ...    ...
1198      일         Noun  문재인      1
1199  하겠습니다         Verb  문재인      1
1200      .  Punctuation  문재인      1
1201  감사합니다         Verb  문재인      1
1202      .  Punctuation  문재인      1

[9264 rows x 4 columns]
##################################################
##################################################
형태소+품사+타입을 기준으로 묶어주기 결과
                         count
terms  pos         type       
!      Punctuation 박근혜       9
                   이명박       9
'      Punctuation 노무현      10
(      Punctuation 이명박       4
)      Punctuation 이명박       4
...                        ...
힘      Noun        이명박       5
힘들었던   Adjective   문재인       1
힘쓰겠습니다 Verb        이명박       2